In [ ]:
using LinearAlgebra
using TensorKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("load_tensor.jl")
include("CTM.jl")
#include("my_TensorKit_fun.jl")
#TensorKit.usebraidcache_nonabelian[] = false

D=12
chi=40
tol=1e-6

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end

global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct];
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct];
end

#TensorKit.usebraidcache_nonabelian[] = false
A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end



global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct];
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct];
end

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;
# display(codomain(PEPS_tensor))
# @time PEPS_tensor1= ncon([bond_tensor,bond_tensor,bond_tensor,triangle_tensor,triangle_tensor],[[-1,1,-5],[4,3,-6],[-4,2,-7],[1,3,2],[4,-2,-3]]);

# @time @tensoropt PEPS_tensor2[-1,-2,-3,-4,-5,-6,-7] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];

# @time @tensoropt PEPS_tensor3[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];


U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor PEPS_tensor[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];




# #PEPS_tensor_dense=convert(Array,PEPS_tensor)

# #save initial CTM to compare with other codes
# @time CTM=init_CTM(10,PEPS_tensor,"PBC");
# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)

type="PBC"
A=PEPS_tensor
display("initialize CTM")
#numind(A)
#numin(A)
#numout(A)
CTM=[]
Cset=Vector(undef,4)
Tset=Vector(undef,4)
#space(A,1)
@time if type=="PBC"
    for direction=1:4    
        inds=(mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),5);
        A_rotate=permute(A,inds)
        Ap_rotate=A_rotate'

        @tensor M[:]:=Ap_rotate[1,-1,-3,2,3]*A_rotate[1,-2,-4,2,3];
        Cset[direction]=M;
        @tensor M[:]:=Ap_rotate[-1,-3,-5,1,2]*A_rotate[-2,-4,-6,1,2];
        Tset[direction]=M
    end

    #fuse legs
    ul_set=Vector(undef,4)
    ur_set=Vector(undef,4)
    for direction=1:2
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4)), space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6)), space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=3:4
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4))', space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6))', space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=1:4
        C=Cset[direction]
        ul=ur_set[mod1(direction-1,4)]
        ur=ul_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Cnew[(-1);(-2)]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]
        @tensor Cnew[:]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Cset[direction]=Cnew

        T=Tset[direction]
        ul=ul_set[direction]
        ur=ur_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Tnew[(-1);(-2,-3,-4)]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]
        @tensor Tnew[:]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Tset[direction]=Tnew
    end
elseif type=="random"
end
CTM=Dict([("Cset", Cset), ("Tset", Tset)])

# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)


In [ ]:
function build_double_layer(A,operator)
    #display(space(A))
    A=permute(A,(1,2,),(3,4,5));
    U_L=unitary(fuse(space(A, 1)' ⊗ space(A, 1)), space(A, 1)' ⊗ space(A, 1));
    U_D=unitary(fuse(space(A, 2)' ⊗ space(A, 2)), space(A, 2)' ⊗ space(A, 2));
    U_R=inv(U_L);
    U_U=inv(U_D);
    # display(space(U_L))
    # display(space(U_D))
    # display(space(U_R))
    # display(space(U_D))

    uM,sM,vM=tsvd(A);
    uM=uM*sM

    uM=permute(uM,(1,2,3,),())
    V=space(vM,1);
    U=unitary(fuse(V' ⊗ V), V' ⊗ V);
    @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
    @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];
    
    vM=permute(vM,(1,2,3,4,),());
    if operator==[]
        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
        @tensor double_RU[:]:=vM'[1,-2,-4,2]*double_RU[-1,1,-3,-5,2];
    else
        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
        @tensor double_RU[:]:=vM'[3,-2,-4,1]*operator[2,1]*double_RU[-1,3,-3,-5,2];
    end
    #display(space(double_RU))

    double_LD=permute(double_LD,(1,2,),(3,4,5,));
    double_LD=U_L*double_LD;
    double_LD=permute(double_LD,(2,3,),(1,4,));
    double_LD=U_D*double_LD;
    double_LD=permute(double_LD,(2,1,),(3,));
    #display(space(double_LD))
    double_RU=permute(double_RU,(1,4,5,),(2,3,));
    double_RU=double_RU*U_R;
    double_RU=permute(double_RU,(1,4,),(2,3,));
    double_RU=double_RU*U_U;
    double_LD=permute(double_LD,(1,2,),(3,));
    double_RU=permute(double_RU,(1,),(2,3,));
    AA_fused=double_LD*double_RU;
    
    return AA_fused, U_L,U_D,U_R,U_U
end


In [ ]:
AA_fused, U_L,U_D,U_R,U_U=build_double_layer(A,[]);


In [ ]:
function fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U)
    #fuse CTM legs
    Tset=CTM["Tset"];

    #T4
    T4=permute(Tset[4],(1,4,),(2,3,));
    T4=T4*U_R;
    T4=permute(T4,(1,3,2,),());
    Tset[4]=T4
    #display(space(T4))

    #T3
    T3=permute(Tset[3],(1,4,),(2,3,));
    T3=T3*U_U;
    T3=permute(T3,(1,3,2,),());
    Tset[3]=T3
    #display(space(T3))

    #T2
    T2=permute(Tset[2],(2,3,),(1,4,));
    T2=U_L*T2;
    T2=permute(T2,(2,1,3,),());
    Tset[2]=T2
    #display(space(T2))

    #T1
    T1=permute(Tset[1],(2,3,),(1,4,));
    T1=U_D*T1;
    T1=permute(T1,(2,1,3,),());
    Tset[1]=T1
    #display(space(T1))

    CTM["Tset"]=Tset;
    return CTM
end


In [ ]:
CTM=fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U);

In [ ]:
function CTM_ite_test(Cset, Tset, AA_fused, chi, direction)
    Tset=CTM["Tset"];
    Cset=CTM["Cset"];

    AA=permute(AA_fused, (mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),),());
                
    @tensor MMup[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-3]*Tset[mod1(direction-1,4)][-1,4,1]*AA[4,-2,-4,3]; 
    @tensor MMlow[:]:=Tset[mod1(direction-1,4)][1,3,-1]*AA[3,4,-4,-2]*Cset[mod1(direction-1,4)][2,1]*Tset[mod1(direction-2,4)][-3,4,2]; 

    @tensor MMup_reflect[:]:=Tset[mod1(direction,4)][-1,3,1]* Cset[mod1(direction+1,4)][1,2]* AA[-2,-4,4,3]* Tset[mod1(direction+1,4)][2,4,-3];
    #@tensor MMlow_reflect[:]:=AA[-2,4,3,-4]*Tset[mod1(direction+1,4)][-3,3,1]*Tset[mod1(direction-2,4)][2,4,-1]*Cset[mod1(direction-2,4)][1,2]; #this is slow compared to other coners, I don't know why            
    @tensor MMlow_reflect[:]:=Tset[mod1(direction+1,4)][-4,-3,2]*Tset[mod1(direction-2,4)][1,-2,-1]*Cset[mod1(direction-2,4)][2,1]; 
    @tensor MMlow_reflect[:]:=MMlow_reflect[-1,1,2,-3]*AA[-2,1,2,-4]; 

    MMup=permute(MMup,(1,2,),(3,4,))

    # _,ss,_=tsvd(MMup)
    # display(convert(Array,ss))

    MMlow=permute(MMlow,(1,2,),(3,4,))
    MMup_reflect=permute(MMup_reflect,(1,2,),(3,4,))
    MMlow_reflect=permute(MMlow_reflect,(1,2,),(3,4,))
    _, RMup=leftorth(permute(MMup*MMup_reflect,(3,4,),(1,2,)));
    _, RMlow=leftorth(permute(MMlow*MMlow_reflect,(3,4,),(1,2,))); 


    RMlow=permute(RMlow,(2,3,),(1,));
    M=RMup*RMlow;



    uM,sM,vM = tsvd(M; trunc=truncdim(chi));
    #uM,sM,vM = tsvd(M; trunc=trunbelow(1e-34)); #this looks to truncate absolute value, not relative value
    #display(convert(Array,sM))
    #display(convert(Array,inv(sM)))
    sM=sM/norm(sM)
    sM_inv=inv(sM);
    sM_dense=convert(Array,sM)
    #display(sM_dense)
    trun_tol=1e-12
    for c1=1:size(sM_dense,1)
        if sM_dense[c1,c1]<trun_tol
            sM_dense[c1,c1]=0;
        end
    end
    #display(sM_dense)
    #display(pinv.(sM_dense))

    #display(sM_inv)
    #display(convert(Array,sM_inv))
    sM_inv_sqrt=sqrt.(convert(Array,sM_inv))
    #display(space(sM_inv))
    #display(sM_inv_sqrt)
    sM_inv_sqrt=TensorMap(pinv.(sqrt.(sM_dense)),codomain(sM_inv)←domain(sM_inv))

    PM_inv=RMlow*vM'*sM_inv_sqrt; 
    PM=sM_inv_sqrt*uM'*RMup; 
    PM=permute(PM,(2,3,),(1,));

    @tensor M5tem[:]:=Tset[mod1(direction-1,4)][4,3,1]*AA[3,5,-2,2]* PM_inv[4,5,-1]* PM[1,2,-3];
    @tensor M1tem[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-2]*PM_inv[1,3,-1];
    @tensor M7tem[:]:=Cset[mod1(direction-1,4)][1,2]*Tset[mod1(direction-2,4)][-1,3,1]* PM[2,3,-2];

    Cset[mod1(direction,4)]=M1tem/norm(M1tem);
    Tset[mod1(direction-1,4)]=M5tem/norm(M5tem);
    Cset[mod1(direction-1,4)]=M7tem/norm(M7tem);
    return Cset,Tset
end

In [ ]:

    #Ref: PHYSICAL REVIEW B 98, 235148 (2018)
    #initial corner transfer matrix

    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    conv_check="singular_value"

    ss_old=ones(chi)*2;
    d=2;
    rho_old=Matrix(I,d^3,d^3);

    #Iteration
    CTM_ite_nums=50;
    display("start CTM iterations:")
    for ci=1:CTM_ite_nums
        direction_order=[1,2,3,4];
        @time for direction in direction_order
            Cset,Tset=CTM_ite_test(Cset, Tset, AA_fused, chi, direction);
        end
        if conv_check=="singular_value" #check convergence of singular value
            _,ss_new,_=svd(permute(Cset[1],(1,),(2,)));
            ss_new=convert(Array,ss_new);
            ss_new=sort(diag(ss_new), rev=true);
            ss_old=ss_old/ss_old[1];
            ss_new=ss_new/ss_new[1];
            #display(ss_new)
            if length(ss_old)>length(ss_new)
                dss=copy(ss_old);
                siz=length(ss_new)
            elseif length(ss_old)<=length(ss_new)
                dss=copy(ss_new);
                siz=length(ss_old)
            end
            dss[1:siz]=ss_old[1:siz]-ss_new[1:siz]
            er=norm(dss);
            display("CTMRG iteration: "*string(ci)*", CTMRG err: "*string(er));
            if er<tol
                break;
            end
            ss_old=ss_new;
        elseif conv_check=="density_matrix" #check reduced density matrix
            
            # ob_opts.SiteNumber=1;
            # CTM_tem.Cset=Cset;
            # CTM_tem.Tset=Tset;
            # rho_new=ob_CTMRG(CTM_tem,A,ob_opts).A;
            # er=sum(sum((abs(rho_old-rho_new))));
            # disp(['CTMRG iteration: ',num2str(ci),' CTMRG err: ',num2str(er)]);
            # if er<tol
            #     break;
            # end
            # rho_old=rho_new;
        end
        
        # if ci==CTM_ite_nums
        #     display(er)
        #     warn("CTMRG does not converge: " * string(er));
        # end
    end







In [ ]:
function ob_1site_closed(CTM,AA_fused)
    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    @tensor envL[:]:=Cset[1][1,-1]*Tset[4][2,-2,1]*Cset[4][-3,2];
    @tensor envR[:]:=Cset[2][-1,1]*Tset[2][1,-2,2]*Cset[3][2,-3];
    @tensor envL[:]:=envL[1,2,4]*Tset[1][1,3,-1]*AA_fused[2,5,-2,3]*Tset[3][-3,5,4];
    @tensor Norm[:]:=envL[1,2,3]*envR[1,2,3];
    Norm=blocks(Norm)[Irrep[SU₂](0)];
    return Norm;
end



In [ ]:
function ob_2sites_x(CTM,AA1,AA2)
    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    @tensor envL[:]:=Cset[1][1,-1]*Tset[4][2,-2,1]*Cset[4][-3,2];
    @tensor envR[:]:=Cset[2][-1,1]*Tset[2][1,-2,2]*Cset[3][2,-3];
    @tensor envL[:]:=envL[1,2,4]*Tset[1][1,3,-1]*AA1[2,5,-2,3,-4]*Tset[3][-3,5,4];
    @tensor envR[:]:=Tset[1][-1,3,1]*AA2[-2,5,2,3,-4]*Tset[3][4,5,-3]*envR[1,2,4];
    @tensor rho[:]:=envL[1,2,3,-1]*envR[1,2,3,-2];
    return rho;
end


In [ ]:

function ob_2sites_y(CTM,AA1,AA2)
    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    @tensor envU[:]:=Cset[2][1,-1]*Tset[1][2,-2,1]*Cset[1][-3,2];
    @tensor envD[:]:=Cset[3][-1,1]*Tset[3][1,-2,2]*Cset[4][2,-3];
    @tensor envU[:]:=envU[1,2,4]*Tset[2][1,3,-1]*AA1[5,-2,3,2,-4]*Tset[4][-3,5,4];
    @tensor envD[:]:=Tset[2][-1,3,1]*AA2[5,2,3,-2,-4]*Tset[4][4,5,-3]*envD[1,2,4];
    @tensor rho[:]:=envU[1,2,3,-1]*envD[1,2,3,-2];
    return rho;
end



In [ ]:
function ob_LD_RU(CTM,AA_fused,AA_LD,AA_RU)
    Cset=CTM["Cset"];
    Tset=CTM["Tset"];

    @tensor MM_LU[:]:=Cset[1][1,2]*Tset[1][2,3,-3]*Tset[4][-1,4,1]*AA_fused[4,-2,-4,3]; 
    @tensor MM_RU[:]:=Tset[1][-1,3,1]* Cset[2][1,2]* AA_RU[-2,-4,4,3,-5]* Tset[2][2,4,-3];

    @tensor MM_LD[:]:=Tset[4][1,3,-2]*AA_LD[3,4,-5,-3,-1]*Cset[4][2,1]*Tset[3][-4,4,2]; 
    @tensor MM_RD[:]:=Tset[2][-4,-3,2]*Tset[3][1,-2,-1]*Cset[3][2,1]; 
    @tensor MM_RD[:]:=MM_RD[-1,1,2,-3]*AA_fused[-2,1,2,-4]; 

    MM_LU=permute(MM_LU,(1,2,),(3,4,));
    MM_RU=permute(MM_RU,(1,2,),(3,4,5,));
    MM_LD=permute(MM_LD,(1,2,3,),(4,5,));
    MM_RD=permute(MM_RD,(1,2,),(3,4,));

    up=MM_LU*MM_RU;
    down=MM_LD*MM_RD;
    @tensor rho[:]:=up[1,2,3,4,-1]*down[-2,1,2,3,4];
end


In [ ]:
function ob_LU_RD(CTM,AA_fused,AA_LU,AA_RD)
    Cset=CTM["Cset"];
    Tset=CTM["Tset"];

    @tensor MM_LU[:]:=Cset[1][1,2]*Tset[1][2,3,-4]*Tset[4][-2,4,1]*AA_LU[4,-3,-5,3,-1]; 
    @tensor MM_RU[:]:=Tset[1][-1,3,1]* Cset[2][1,2]* AA_fused[-2,-4,4,3]* Tset[2][2,4,-3];

    @tensor MM_LD[:]:=Tset[4][1,3,-1]*AA_fused[3,4,-4,-2]*Cset[4][2,1]*Tset[3][-3,4,2]; 
    @tensor MM_RD[:]:=Tset[2][-4,-3,2]*Tset[3][1,-2,-1]*Cset[3][2,1]; 
    @tensor MM_RD[:]:=MM_RD[-1,1,2,-3]*AA_RD[-2,1,2,-4,-5]; 

    MM_LU=permute(MM_LU,(1,2,3,),(4,5,));
    MM_RU=permute(MM_RU,(1,2,),(3,4,));
    MM_LD=permute(MM_LD,(1,2,),(3,4,));
    MM_RD=permute(MM_RD,(1,2,),(3,4,5,));

    up=MM_LU*MM_RU;
    down=MM_LD*MM_RD;
    @tensor rho[:]:=up[-1,1,2,3,4]*down[1,2,3,4,-2];
end


In [ ]:
Norm=ob_1site_closed(CTM,AA_fused);


In [ ]:
# Heisenberg interaction
Id=I(2);
sx=[[0,1] [1,0]]/2; sy=[[0,1] [-1,0]]/2*im; sz=[[1,0] [0,-1]]/2;
@tensor H12[:]:=sx[-1,-4]*sx[-2,-5]*Id[-3,-6]+sy[-1,-4]*sy[-2,-5]*Id[-3,-6]+sz[-1,-4]*sz[-2,-5]*Id[-3,-6];
@tensor H31[:]:=sx[-1,-4]*Id[-2,-5]*sx[-3,-6]+sy[-1,-4]*Id[-2,-5]*sy[-3,-6]+sz[-1,-4]*Id[-2,-5]*sz[-3,-6];
@tensor H23[:]:=Id[-1,-4]*sx[-2,-5]*sx[-3,-6]+Id[-1,-4]*sy[-2,-5]*sy[-3,-6]+Id[-1,-4]*sz[-2,-5]*sz[-3,-6];
H12_tensorkit=TensorMap(H12, domain(U_phy) ← domain(U_phy));
H31_tensorkit=TensorMap(H31, domain(U_phy) ← domain(U_phy));
H23_tensorkit=TensorMap(H23, domain(U_phy) ← domain(U_phy));
@tensor H12_tensorkit[:]:=U_phy'[4,5,6,-1]*H12_tensorkit[1,2,3,4,5,6]*U_phy[-2,1,2,3];
@tensor H31_tensorkit[:]:=U_phy'[4,5,6,-1]*H31_tensorkit[1,2,3,4,5,6]*U_phy[-2,1,2,3];
@tensor H23_tensorkit[:]:=U_phy'[4,5,6,-1]*H23_tensorkit[1,2,3,4,5,6]*U_phy[-2,1,2,3];

@tensor H[:]:=sx[-1,-3]*sx[-2,-4]+sy[-1,-3]*sy[-2,-4]+sz[-1,-3]*sz[-2,-4];


In [ ]:

#calculate single unit-cell observable
AA_12_fused, _,_,_,_=build_double_layer(A,H12_tensorkit);
ob12=ob_1site_closed(CTM,AA_12_fused)/Norm;
AA_31_fused, _,_,_,_=build_double_layer(A,H31_tensorkit);
ob31=ob_1site_closed(CTM,AA_31_fused)/Norm;
AA_23_fused, _,_,_,_=build_double_layer(A,H23_tensorkit);
ob23=ob_1site_closed(CTM,AA_23_fused)/Norm;
E=ob12+ob31+ob23;
display("energy is: "*string(E/3*2))


In [ ]:
# function build_double_layer_open_test(A,inds,U_phy,U_L,U_D,U_R,U_U)
#     #display(space(A))
#     A=permute(A,(1,2,3,),(4,5,));
#     V_D=space(A, 4);
#     V_DD=fuse(V_D' ⊗ V_D);

#     #seperate the physical index that to be open
#     if inds=="123"
#         V_d=space(A, 5);
#         V_D_d=V_D ⊗ V_d;
#         V_Dd=fuse(V_D_d);
#         V_dd=fuse(V_d' ⊗ V_d);
#         U_Dd=unitary(V_Dd,V_D_d);
#         @tensor A_fused[:]:=A[-1,-2,-3,1,2]*U_Dd[-4,1,2];

#         A_fused=permute(A_fused,(1,2,),(3,4,));
#         uM,sM,vM=tsvd(A_fused);
#         uM=uM*sM
    
#         uM=permute(uM,(1,2,3,),());
#         V=space(vM,1);
#         U=unitary(fuse(V' ⊗ V), V' ⊗ V);
#         @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
#         @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

#         vM=permute(vM,(1,2,3,),());
#         @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4];
#         @tensor double_RU[:]:=vM'[1,-2,-4]*double_RU[-1,1,-3,-5];

#         double_LD=permute(double_LD,(1,2,),(3,4,5,));
#         double_LD=U_L*double_LD;
#         double_LD=permute(double_LD,(2,3,),(1,4,));
#         double_LD=U_D*double_LD;
#         double_LD=permute(double_LD,(2,1,),(3,));
    
#         double_RU=permute(double_RU,(1,4,5,),(2,3,));
#         double_RU=double_RU*U_R;
#         double_RU=permute(double_RU,(1,4,),(2,3,));
    
#         U_Ud=unitary(space(U_U,3) ⊗ V_dd, V_Dd' ⊗ V_Dd);
#         @tensor double_RU[:]:=double_RU[-1,-2,1,2]*U_Ud[-3,-4,1,2];
#         double_RU=permute(double_RU,(1,),(2,3,4,));
#         AA_open_fused=permute(double_LD*double_RU,(1,2,3,4,5),());
        
#         U_dd=unitary(V_d' ⊗ V_d, V_dd);
#         return AA_open_fused, U_dd
#     elseif inds in ["1","2","3"]
#         V_s=space(U_phy,4)';
#         U_phy_sep=inv(U_phy);
#         if inds=="1"
#         elseif inds=="2"
#             U_phy_sep=permute(U_phy_sep,(2,1,3,),(4,));
#         elseif inds=="3"
#             U_phy_sep=permute(U_phy_sep,(3,1,2,),(4,));
#         end
#         U_phy_sep=unitary(space(U_phy_sep,1) ⊗ fuse(space(U_phy_sep,2) ⊗ space(U_phy_sep,3)), codomain(U_phy_sep))*U_phy_sep;
#         @tensor A_fused[:]:=A[-1,-2,-3,-4,1]*U_phy_sep[-5,-6,1];
#         display(space(U_phy_sep))
#         V_D_s=V_D ⊗ V_s;
#         V_Ds=fuse(V_D_s);
#         V_ss=fuse(V_s' ⊗ V_s);
#         U_Ds=unitary(V_Ds,V_D_s);
#         display(space(A_fused))
#         @tensor A_fused[:]:=A_fused[-1,-2,-3,1,2,-5]*U_Ds[-4,1,2];

#         A_fused=permute(A_fused,(1,2,),(3,4,5,));
#         uM,sM,vM=tsvd(A_fused);
#         uM=uM*sM
    
#         uM=permute(uM,(1,2,3,),());
#         V=space(vM,1);
#         U=unitary(fuse(V' ⊗ V), V' ⊗ V);
#         @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
#         @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

#         vM=permute(vM,(1,2,3,4,),());
#         @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5];
#         @tensor double_RU[:]:=vM'[1,-2,-4,2]*double_RU[-1,1,-3,-5,2];

#         double_LD=permute(double_LD,(1,2,),(3,4,5,));
#         double_LD=U_L*double_LD;
#         double_LD=permute(double_LD,(2,3,),(1,4,));
#         double_LD=U_D*double_LD;
#         double_LD=permute(double_LD,(2,1,),(3,));
    
#         double_RU=permute(double_RU,(1,4,5,),(2,3,));
#         double_RU=double_RU*U_R;
#         double_RU=permute(double_RU,(1,4,),(2,3,));
    
#         U_Ds_inv=inv(U_Ds);
#         @tensor double_RU[:]:=U_Ds_inv'[1,-3,-5]*double_RU[-1,-2,1,2]*U_Ds_inv[-4,-6,2];
#         @tensor double_RU[:]:=double_RU[-1,-2,1,2,-4,-5]*U_U[1,2,-3];
#         display(space(double_RU))
#         U_s_s=unitary(V_ss, V_s' ⊗ V_s);
#         @tensor double_RU[:]:=double_RU[-1,-2,-3,1,2]*U_s_s[-4,1,2];
#         double_RU=permute(double_RU,(1,),(2,3,4,));
#         AA_open_fused=permute(double_LD*double_RU,(1,2,3,4,5),());
        
#         return AA_open_fused, U_s_s'

#     end
# end


In [ ]:
# function build_double_layer_open_test(A_unfused,inds,U_phy,U_L,U_D,U_R,U_U)


#     #seperate the physical index that to be open
#     if inds=="123"
#         A=permute(A,(1,2,3,),(4,5,));
#         V_D=space(A, 4);
#         V_DD=fuse(V_D' ⊗ V_D);
#         V_d=space(A, 5);
#         V_D_d=V_D ⊗ V_d;
#         V_Dd=fuse(V_D_d);
#         V_dd=fuse(V_d' ⊗ V_d);
#         U_Dd=unitary(V_Dd,V_D_d);
#         @tensor A_fused[:]:=A[-1,-2,-3,1,2]*U_Dd[-4,1,2];

#         A_fused=permute(A_fused,(1,2,),(3,4,));
#         uM,sM,vM=tsvd(A_fused);
#         uM=uM*sM
    
#         uM=permute(uM,(1,2,3,),());
#         V=space(vM,1);
#         U=unitary(fuse(V' ⊗ V), V' ⊗ V);
#         @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
#         @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

#         vM=permute(vM,(1,2,3,),());
#         @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4];
#         @tensor double_RU[:]:=vM'[1,-2,-4]*double_RU[-1,1,-3,-5];

#         double_LD=permute(double_LD,(1,2,),(3,4,5,));
#         double_LD=U_L*double_LD;
#         double_LD=permute(double_LD,(2,3,),(1,4,));
#         double_LD=U_D*double_LD;
#         double_LD=permute(double_LD,(2,1,),(3,));
    
#         double_RU=permute(double_RU,(1,4,5,),(2,3,));
#         double_RU=double_RU*U_R;
#         double_RU=permute(double_RU,(1,4,),(2,3,));
    
#         U_Ud=unitary(space(U_U,3) ⊗ V_dd, V_Dd' ⊗ V_Dd);
#         @tensor double_RU[:]:=double_RU[-1,-2,1,2]*U_Ud[-3,-4,1,2];
#         double_RU=permute(double_RU,(1,),(2,3,4,));
#         AA_open_fused=permute(double_LD*double_RU,(1,2,3,4,5),());
        
#         U_dd=unitary(V_d' ⊗ V_d, V_dd);
#         return AA_open_fused, U_dd
#     elseif inds in ["1","2","3"]
#         #display(space(A))
#         #A=permute(A,(1,2,3,),(4,5,));
#         V_D=space(A_unfused, 4);
#         V_s=space(A_unfused,7);

#         if inds=="1"
#             A_unfused=permute(A_unfused,(1,2,3,4,5,6,7,),());
#         elseif inds=="2"
#             A_unfused=permute(A_unfused,(1,2,3,4,6,5,7,),());
#         elseif inds=="3"
#             A_unfused=permute(A_unfused,(1,2,3,4,7,5,6,),());
#         end
#         fuse_spin=unitary(fuse(V_s ⊗ V_s), V_s ⊗ V_s);
#         @tensor A_fused[:]:=A_unfused[-1,-2,-3,-4,-5,1,2]*fuse_spin[-6,1,2];


#         V_ss=fuse(V_s' ⊗ V_s);

#         A_fused=permute(A_fused,(1,2,),(3,4,5,6,));
#         uM,sM,vM=tsvd(A_fused);
#         uM=uM*sM
    
#         uM=permute(uM,(1,2,3,),());
#         V=space(vM,1);
#         U=unitary(fuse(V' ⊗ V), V' ⊗ V);
#         @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
#         @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

#         vM=permute(vM,(1,2,3,4,5,),());
#         @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5,-6];
#         @tensor double_RU[:]:=vM'[1,-2,-4,-6,2]*double_RU[-1,1,-3,-5,-7,2];

#         double_LD=permute(double_LD,(1,2,),(3,4,5,));
#         double_LD=U_L*double_LD;
#         double_LD=permute(double_LD,(2,3,),(1,4,));
#         double_LD=U_D*double_LD;
#         double_LD=permute(double_LD,(2,1,),(3,));
    
#         double_RU=permute(double_RU,(1,4,5,6,7,),(2,3,));
#         double_RU=double_RU*U_R;
#         double_RU=permute(double_RU,(1,6,4,5,),(2,3,));
#         @tensor double_RU[:]:=double_RU[-1,-2,-4,-5,1,2]*U_U[1,2,-3];
#         U_s_s=unitary(V_ss, V_s' ⊗ V_s);
#         @tensor double_RU[:]:=double_RU[-1,-2,-3,1,2]*U_s_s[-4,1,2];
#         double_RU=permute(double_RU,(1,),(2,3,4,));
#         AA_open_fused=permute(double_LD*double_RU,(1,2,3,4,5),());
        
#         return AA_open_fused, U_s_s'

#     end
# end


In [ ]:
function build_double_layer_open_test(A_unfused,inds,U_phy,U_L,U_D,U_R,U_U)


    #seperate the physical index that to be open
    if inds=="123"
        # A=permute(A,(1,2,3,),(4,5,));
        # V_D=space(A, 4);
        # V_DD=fuse(V_D' ⊗ V_D);
        # V_d=space(A, 5);
        # V_D_d=V_D ⊗ V_d;
        # V_Dd=fuse(V_D_d);
        # V_dd=fuse(V_d' ⊗ V_d);
        # U_Dd=unitary(V_Dd,V_D_d);
        # @tensor A_fused[:]:=A[-1,-2,-3,1,2]*U_Dd[-4,1,2];

        # A_fused=permute(A_fused,(1,2,),(3,4,));
        # uM,sM,vM=tsvd(A_fused);
        # uM=uM*sM
    
        # uM=permute(uM,(1,2,3,),());
        # V=space(vM,1);
        # U=unitary(fuse(V' ⊗ V), V' ⊗ V);
        # @tensor double_LD[:]:=uM'[-1,-2,1]*U'[1,-3,-4];
        # @tensor double_LD[:]:=double_LD[-1,-3,1,-5]*uM[-2,-4,1];

        # vM=permute(vM,(1,2,3,),());
        # @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4];
        # @tensor double_RU[:]:=vM'[1,-2,-4]*double_RU[-1,1,-3,-5];

        # double_LD=permute(double_LD,(1,2,),(3,4,5,));
        # double_LD=U_L*double_LD;
        # double_LD=permute(double_LD,(2,3,),(1,4,));
        # double_LD=U_D*double_LD;
        # double_LD=permute(double_LD,(2,1,),(3,));
    
        # double_RU=permute(double_RU,(1,4,5,),(2,3,));
        # double_RU=double_RU*U_R;
        # double_RU=permute(double_RU,(1,4,),(2,3,));
    
        # U_Ud=unitary(space(U_U,3) ⊗ V_dd, V_Dd' ⊗ V_Dd);
        # @tensor double_RU[:]:=double_RU[-1,-2,1,2]*U_Ud[-3,-4,1,2];
        # double_RU=permute(double_RU,(1,),(2,3,4,));
        # AA_open_fused=permute(double_LD*double_RU,(1,2,3,4,5),());
        
        # U_dd=unitary(V_d' ⊗ V_d, V_dd);
        # return AA_open_fused, U_dd
    elseif inds in ["1","2","3"]
        #display(space(A))
        #A=permute(A,(1,2,3,),(4,5,));
        V_D=space(A_unfused, 4);
        V_s=space(A_unfused,7);

        if inds=="1"
            A_unfused=permute(A_unfused,(1,2,3,4,5,6,7,),());
        elseif inds=="2"
            A_unfused=permute(A_unfused,(1,2,3,4,6,5,7,),());
        elseif inds=="3"
            A_unfused=permute(A_unfused,(1,2,3,4,7,5,6,),());
        end
        fuse_spin=unitary(fuse(V_s ⊗ V_s), V_s ⊗ V_s);
        @tensor A_fused[:]:=A_unfused[-1,-2,-3,-4,-5,1,2]*fuse_spin[-6,1,2];
        V_ss=fuse(V_s' ⊗ V_s);

        uM_dag,sM_dag,vM_dag=tsvd(permute(A_fused',(1,2,3,),(4,5,6,)));
        uM_dag=uM_dag*sM_dag;
        uM,sM,vM=tsvd(permute(A_fused,(1,2,),(3,4,5,6,)));
        uM=uM*sM;
        
        uM_dag=permute(uM_dag,(1,2,3,4,),());
        uM=permute(uM,(1,2,3,),());
        Vp=space(vM_dag,1);
        V=space(vM,1);
        U=unitary(fuse(Vp ⊗ V), Vp ⊗ V);
        @tensor double_LD[:]:=uM_dag[-1,-2,-3,1]*U'[1,-4,-5];
        @tensor double_LD[:]:=double_LD[-1,-3,-5,1,-6]*uM[-2,-4,1];

        vM_dag=permute(vM_dag,(1,2,3,4,),());
        vM=permute(vM,(1,2,3,4,5,),());
        @tensor double_RU[:]:=U[-1,-2,1]*vM[1,-3,-4,-5,-6];
        @tensor double_RU[:]:=vM_dag[1,-3,-5,2]*double_RU[-1,1,-2,-4,-6,2];

        double_LD=permute(double_LD,(1,2,),(3,4,5,6,));
        double_LD=U_L*double_LD;
        double_LD=permute(double_LD,(2,3,),(1,4,5,));
        double_LD=U_D*double_LD;
        double_LD=permute(double_LD,(2,1,3,4),());
    
        double_RU=permute(double_RU,(1,2,5,6,),(3,4,));
        double_RU=double_RU*U_U;
        U_s_s=unitary(V_ss, V_s' ⊗ V_s);
        @tensor double_RU[:]:=double_RU[-1,-2,1,2,-3]*U_s_s[-4,1,2];

        A_fused=[];#clear memory
        A_unfused=[];#clear memory
        @tensor double_RU[:]:=double_RU[-1,1,-4,-5]*U_R[-2,1,-3];

        @tensor AA_open_fused[:]:=double_LD[-1,-2,2,1]*double_RU[1,2,-3,-4,-5];

        return AA_open_fused, U_s_s'

    end
end


In [ ]:
AA1, U_ss=build_double_layer_open_test(A_unfused,"1",U_phy,U_L,U_D,U_R,U_U);

In [ ]:


AA1, U_ss=build_double_layer_open_test(A_unfused,"1",U_phy,U_L,U_D,U_R,U_U);
AA2, U_ss=build_double_layer_open_test(A_unfused,"2",U_phy,U_L,U_D,U_R,U_U);
AA3, U_ss=build_double_layer_open_test(A_unfused,"3",U_phy,U_L,U_D,U_R,U_U);


In [ ]:
rhox=ob_2sites_x(CTM,AA2,AA1);
rhoy=ob_2sites_y(CTM,AA2,AA3);
display(space(rhox))


In [ ]:
rho_LD_RU=ob_LD_RU(CTM,AA_fused,AA3,AA1);


In [ ]:

#rho_LU_RD=ob_LU_RD(CTM,AA_fused,AA1,AA2);


In [ ]:
@tensor rhox[:]:=U_ss[-1,-3,1]*U_ss[-2,-4,2]*rhox[1,2];
@tensor rhoy[:]:=U_ss[-1,-3,1]*U_ss[-2,-4,2]*rhoy[1,2];
@tensor rho_LD_RU[:]:=U_ss[-1,-3,1]*U_ss[-2,-4,2]*rho_LD_RU[1,2];


In [ ]:
rhox=convert(Array,rhox);
rhoy=convert(Array,rhoy);
rho_LD_RU=convert(Array,rho_LD_RU);



In [ ]:
@tensor norm_rhox[:]:=rhox[1,2,1,2];
@tensor norm_rhoy[:]:=rhoy[1,2,1,2];
@tensor norm_LD_RU[:]:=rho_LD_RU[1,2,1,2];


In [ ]:
@tensor E_down_21[:]:=rhox[1,2,3,4]*H[3,4,1,2];
E_down_21=E_down_21[1]/norm_rhox[1];
display(E_down_21)

@tensor E_down_23[:]:=rhoy[1,2,3,4]*H[3,4,1,2];
E_down_23=E_down_23[1]/norm_rhoy[1];
display(E_down_23)

@tensor E_down_13[:]:=rho_LD_RU[1,2,3,4]*H[3,4,1,2];
E_down_13=E_down_13[1]/norm_LD_RU[1];
display(E_down_13)

